In [129]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import missingno as msno


In [50]:
world_bank = pd.read_csv("data/SDG_csv/SDGData.csv", sep=",")

In [40]:
life_expectancy = pd.read_csv("data/life-expectancy.csv")

In [45]:
household_ppp = pd.read_excel("data/NHA-indicators.xlsx")

In [100]:
#Zoek machine voor onderwerpen uit de indicator naam. Niet hoofdletter gevoelig. 
#Vergeet niet country_names te laden.

variabelen = []
for sentence in list(world_bank["Indicator Name"].values):
    sentence1 = np.array(sentence.split(" "))
    for words in (sentence1):
        if words.lower() == "GDP".lower(): #Vul hier zoekterm in.
            variabelen += [sentence]

variabelen = pd.unique(variabelen)
print(variabelen)                       
                                


['GDP (constant 2010 US$)' 'GDP (constant LCU)' 'GDP (current LCU)'
 'GDP (current US$)' 'GDP growth (annual %)'
 'GDP per capita (constant 2010 US$)' 'GDP per capita (constant LCU)'
 'GDP per capita (current LCU)' 'GDP per capita (current US$)'
 'GDP per capita growth (annual %)'
 'GDP per capita, PPP (constant 2011 international $)'
 'GDP per capita, PPP (current international $)'
 'GDP per person employed (constant 2011 PPP $)'
 'PPP conversion factor, GDP (LCU per international $)'
 'Water productivity, total (constant 2010 US$ GDP per cubic meter of total freshwater withdrawal)']


In [155]:
def find_top20():
    countries = world_bank[world_bank["Indicator Name"]== 'GDP per capita, PPP (constant 2011 international $)']["Country Name"]
    GDP_2016 = world_bank[world_bank["Indicator Name"]== 'GDP per capita, PPP (constant 2011 international $)']["2016"]
    GDP_2017 = world_bank[world_bank["Indicator Name"]== 'GDP per capita, PPP (constant 2011 international $)']["2017"]

    GDP = countries.to_frame().join(GDP_2017.to_frame()).join(GDP_2016.to_frame())
    #Mask the area's which come up at the highest and lowest countries and are undesired
    GDP = GDP.mask(GDP["Country Name"]=="Macao SAR, China")
    GDP = GDP.mask(GDP["Country Name"]=="Hong Kong SAR, China")
    GDP = GDP.mask(GDP["Country Name"]=="North America")

    lowest_20 = GDP.sort_values("2017")[0:20]
    highest_20 = GDP.sort_values("2017", ascending=False,)[0:20]

    lowest_countries = lowest_20["Country Name"].values
    highest_countries = highest_20["Country Name"].values
    if (lowest_countries.count < 19):
        return false
    else:
        return true
    




In [154]:
#msno.matrix(world_bank.sample(len(world_bank)))

